In [11]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import torch
from datetime import datetime, timezone
import logging

logging.basicConfig(
    level=logging.INFO,  # Set the logging level
    format='%(asctime)s - %(levelname)s - %(message)s',  # Format for the log messages
    handlers=[
        logging.StreamHandler()  # Log to the console
    ]
)

%reload_ext autoreload
%autoreload 2
from data.raw.retrievers.alpaca_markets_retriever import AlpacaMarketsRetriever
from data.raw.retrievers.stooq_retriever import StooqRetriever

from config.constants import *
from data.processed.dataset_creation import DatasetCreator
from data.processed.indicators import *
from data.processed.targets import Balanced3ClassClassification
from data.processed.normalization import ZScoreOverWindowNormalizer, ZScoreNormalizer, MinMaxNormalizer
from data.processed.dataset_pytorch import DatasetPytorch
from modeling.trainer import Trainer
from modeling.evaluate import evaluate_lgb_regressor, evaluate_torch_regressor, evaluate_torch_regressor_multiasset
# from observability.mlflow_integration import log_experiment

from modeling.rl.environment import PortfolioEnvironment
from modeling.rl.state import State
from modeling.rl.agent import RlAgent
from modeling.rl.algorithms.policy_gradient import PolicyGradient
from modeling.rl.actors.actor import RlActor, FullyConnectedBackend, TransformerBackend
from modeling.rl.actors.signal_predictor_actor import SignalPredictorActor
from modeling.rl.actors.high_energy_low_friction_actor import HighEnergyLowFrictionActor
from modeling.rl.actors.xsmom_actor import XSMomActor
from modeling.rl.actors.tsmom_actor import TSMomActor
from modeling.rl.actors.blsw_actor import BLSWActor
from modeling.rl.trajectory_dataset import TrajectoryDataset
from modeling.rl.metrics import MetricsCalculator, DEFAULT_METRICS
from modeling.rl.reward import EstimatedReturnReward
from modeling.rl.loss import SumLogReturnLoss
from modeling.rl.visualization.wealth_plot import plot_cumulative_wealth
from modeling.rl.visualization.position_plot import plot_position_heatmap
from config.experiments.cur_experiment import config

torch.backends.cudnn.benchmark = config.train_config.cudnn_benchmark

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [12]:
str(config.data_config.frequency)

'1Day'

In [ ]:
retriever = StooqRetriever(download_from_gdrive=False)


2025-08-18 16:51:50,863 - INFO - Downloading folder djia from https://drive.google.com/drive/folders/14cvQhyjltHfxa_-rVrGi67QfrsYtWqHh?usp=sharing to ../data/raw/stooq/bars_test
Retrieving folder contents


Processing file 1bmC97stqjcKKBjEpglgfO62pReoo53gN aapl_us_d.csv
Processing file 1zwnEVdC5iXTaHNs8JDtjrVk80dsptMxx amgn_us_d.csv
Processing file 1oY9_Jf7_JKAi_kJjxzTf-Kh7FtJk5cWJ amzn_us_d.csv
Processing file 1uighjCda3iEd0yfyHIUO-T2ErkKfvf77 axp_us_d.csv
Processing file 1exVHwzwKTj6kCRqa8xhuggxRBon15nQo ba_us_d.csv
Processing file 1Vv8A1JvEmae-H8Mlh2lE2IMR1h6_LZqb cat_us_d.csv
Processing file 1qQytT6Nk_jT7Rux41IpnD98q5dF5xBgh crm_us_d.csv
Processing file 1z4cOP0UjiF_5YBfEUk3qPKJqKWvtbS6x csco_us_d.csv
Processing file 1qfwPlFCeJucgx1g26Z0qYWb1bHmY1aWp cvx_us_d.csv
Processing file 1jN2lH25A6Xq5-OB7bJKxdgbn8-iMUAgR dis_us_d.csv
Processing file 1JWtiD9oAaDwwWdTBr8iA8se1PfTjkU5I gs_us_d.csv
Processing file 1Hg7_JXL0Q-3fPPNJY1ASqTZ8zmWuXe3w hd_us_d.csv
Processing file 1_4_KlWqUBSId4n9Iux4Wa59fd6NuYN-Z hon_us_d.csv
Processing file 1htoKNQM-wMJW_RqcgDI3Yxcm8HPovK3h ibm_us_d.csv
Processing file 1eSLDJTSNbs5PmgxYXr0zoyM6ok_JPDFg jnj_us_d.csv
Processing file 1w8UFAXtzl1_21oz5lGj19wQrEiNIODGV jpm_

Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1bmC97stqjcKKBjEpglgfO62pReoo53gN
To: c:\Users\ikurnosau\Projects\QuantitativeTrading\intraday-portfolio-management\data\raw\stooq\bars_test\djia\aapl_us_d.csv
100%|██████████| 573k/573k [00:00<00:00, 6.79MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zwnEVdC5iXTaHNs8JDtjrVk80dsptMxx
To: c:\Users\ikurnosau\Projects\QuantitativeTrading\intraday-portfolio-management\data\raw\stooq\bars_test\djia\amgn_us_d.csv
100%|██████████| 611k/611k [00:00<00:00, 5.51MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oY9_Jf7_JKAi_kJjxzTf-Kh7FtJk5cWJ
To: c:\Users\ikurnosau\Projects\QuantitativeTrading\intraday-portfolio-management\data\raw\stooq\bars_test\djia\amzn_us_d.csv
100%|██████████| 345k/345k [00:00<00:00, 4.85MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uighjCda3iEd0yfyHIUO-T2ErkKfvf77
To: c:\Users\ikurnos

In [ ]:
def evaluate_signal_predictor(start_date: datetime, end_date: datetime, train_set_last_date: datetime):
    retrieval_result = retriever.bars(start=start_date, end=end_date)

    dataset_creator = DatasetCreator(
        features=config.data_config.features,
        target=config.data_config.target,
        normalizer=config.data_config.normalizer,
        missing_values_handler=config.data_config.missing_values_handler,
        train_set_last_date=train_set_last_date, 
        cutoff_time=config.data_config.cutoff_time,
        in_seq_len=config.data_config.in_seq_len,
        multi_asset_prediction=config.data_config.multi_asset_prediction,
    )

    X_train, y_train, next_return_train, spread_train, volatility_train, X_test, y_test, next_return_test, spread_test, volatility_test = dataset_creator.create_dataset_numpy(retrieval_result)

    train_loader = DatasetPytorch(X_train, y_train, learning_task='regression').as_dataloader(
        batch_size=config.train_config.batch_size,
        shuffle=config.train_config.shuffle,
        num_workers=config.train_config.num_workers,
        prefetch_factor=config.train_config.prefetch_factor,
        pin_memory=config.train_config.pin_memory,
        persistent_workers=config.train_config.persistent_workers,
        drop_last=config.train_config.drop_last
    )
    test_loader = DatasetPytorch(X_test, y_test, learning_task='regression').as_dataloader(
        batch_size=config.train_config.batch_size,
        shuffle=config.train_config.shuffle,
        num_workers=config.train_config.num_workers,
        prefetch_factor=config.train_config.prefetch_factor,
        pin_memory=config.train_config.pin_memory,
        persistent_workers=config.train_config.persistent_workers,
        drop_last=config.train_config.drop_last
    )

    model = config.model_config.model

    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=test_loader,
        loss_fn=config.train_config.loss_fn,
        optimizer=config.train_config.optimizer,
        scheduler=config.train_config.scheduler,
        num_epochs=config.train_config.num_epochs,
        device=config.train_config.device,
        metrics=config.train_config.metrics,
        save_path=config.train_config.save_path
    )

    model, history = trainer.train()

    val_trajectory_loader = TrajectoryDataset(X_test, next_return_test, spread_test, volatility_test, trajectory_length=10).as_dataloader(
        batch_size=8, 
        shuffle=False,
        num_workers=8,
        prefetch_factor=4,
        pin_memory=True,
        persistent_workers=True,
        drop_last=True,
    )

    env = PortfolioEnvironment(
        reward_function=EstimatedReturnReward(fee=0.001, spread_multiplier=0.0),
    )

    backend = FullyConnectedBackend(
        n_assets=len(config.data_config.symbol_or_symbols),
        hidden_dim=128,
        num_layers=2, 
        dropout=0.1,
        use_layer_norm=False,
    )

    actor = RlActor(
        model, 
        backend,
        n_assets=len(config.data_config.symbol_or_symbols),
        train_signal_predictor=False, 
        exploration_eps=0.0
    ).to(device)

    signal_predictor_actor = SignalPredictorActor(
        model, 
        trade_asset_count=1,
        train_signal_predictor=False
    ).to(device)

    rl_agent = RlAgent(
        actor, 
        env,
        single_action_per_trajectory=True
    )

    metrics_calculator = MetricsCalculator(
        metrics=DEFAULT_METRICS
    )

    policy_gradient = PolicyGradient(
        rl_agent, 
        None, 
        val_trajectory_loader, 
        metrics_calculator=metrics_calculator,
        optimizer=torch.optim.AdamW(
            [p for p in actor.parameters() if p.requires_grad], 
            lr=1e-3,
            weight_decay=1e-5,
            amsgrad=True),
        scheduler=None,
        loss_fn=SumLogReturnLoss(use_baseline=False),
        num_epochs=10,
        device=device
    )

    epoch_loss, realized_returns_signal_predictor, actions_signal_predictor = policy_gradient.evaluate(signal_predictor_actor)
    return realized_returns_signal_predictor


In [4]:
start_end_last_train_dates = [
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2008, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2007, 9, 1, tzinfo=timezone.utc)},
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2009, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2008, 9, 1, tzinfo=timezone.utc)},
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2010, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2009, 9, 1, tzinfo=timezone.utc)},
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2011, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2010, 9, 1, tzinfo=timezone.utc)},
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2012, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2011, 9, 1, tzinfo=timezone.utc)},
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2013, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2012, 9, 1, tzinfo=timezone.utc)},
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2014, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2013, 9, 1, tzinfo=timezone.utc)},
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2015, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2014, 9, 1, tzinfo=timezone.utc)},
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2016, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2015, 9, 1, tzinfo=timezone.utc)},
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2017, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2016, 9, 1, tzinfo=timezone.utc)},
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2018, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2017, 9, 1, tzinfo=timezone.utc)},
    {'start': datetime(1999, 6, 1, tzinfo=timezone.utc), 'end': datetime(2019, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2018, 9, 1, tzinfo=timezone.utc)},
]

In [5]:
realized_returns = []
for i in range(len(start_end_last_train_dates)):
    realized_returns.extend(
        evaluate_signal_predictor(
            start_end_last_train_dates[i]['start'], 
            start_end_last_train_dates[i]['end'], 
            start_end_last_train_dates[i]['last_train_date']))

c:\Users\ikurnosau\Projects\QuantitativeTrading\intraday-portfolio-management\data\raw\retrievers\stooq_retriever.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  asset_dfs[name.split('_')[0].upper()] = df[df['date'] >= start][df['date'] <= end]
c:\Users\ikurnosau\Projects\QuantitativeTrading\intraday-portfolio-management\data\raw\retrievers\stooq_retriever.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  asset_dfs[name.split('_')[0].upper()] = df[df['date'] >= start][df['date'] <= end]
c:\Users\ikurnosau\Projects\QuantitativeTrading\intraday-portfolio-management\data\raw\retrievers\stooq_retriever.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  asset_dfs[name.split('_')[0].upper()] = df[df['date'] >= start][df['date'] <= end]
c:\Users\ikurnosau\Projects\QuantitativeTrading\intraday-portfolio-management\data\raw\retrievers\stooq_retriever.py:34: UserWarning: Boolean Series key w

RuntimeError: DataLoader worker (pid(s) 32464, 28920, 1848, 39128) exited unexpectedly